<a href="https://colab.research.google.com/github/jpdicarvalho/POO-Python/blob/main/Projeto_Elei%C3%A7%C3%B5es_de_Condom%C3%ADnio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Eleição de Condomínio**
# **Author: João Pedro Braga de Carvalho**
---
## **Documentação do Projeto: Organizando uma Eleição**
Este documento descreve a estrutura e a funcionalidade do sistema eletrônico desenvolvido para auxiliar na votação do novo síndico em um condomínio.

#Estrutura de Classes

## Morador
Representa cada pessoa que habita no condomínio.
1. **Atributos**: nome, apartamento.
2. **Métodos**:
- **Construtor**: recebe opcionalmente um nome e um objeto Apartamento. Se não receber um nome, solicita através de input. Se não receber um objeto Apartamento, solicita o número do apartamento e cria o objeto.
- **Votar**: recebe um objeto Urna e um número de candidato (opcional) e registra o voto na Urna.

In [5]:
class Morador():
    def __init__(self, nome, num_apartamento):
        self.nome = nome
        self.num_apartamento = num_apartamento

    def votar(self, urna, num_apartamento):
        num_candidato_votado = input("\nDigite o número do candidato em que deseja votar (ou digite 0 para sair): ")
        if num_candidato_votado == '0':
            return  # Encerra a função se o usuário digitar 0

        num_candidato_votado = int(num_candidato_votado)  # Convertendo para inteiro

        for apartamento in urna.listaApartamentos.values():
            if 'Candidato' in apartamento:
                if apartamento['numCandidatura'] == num_candidato_votado:
                    apartamento['votosRecebidos'] += 1

        quemVotou = urna.listaApartamentos[self.num_apartamento]
        if not quemVotou['voto']:
            quemVotou['voto'] = True
            print("Voto registrado com sucesso!")
            return


        print("Número de candidatura não encontrado. Por favor, tente novamente.")

## Candidato
É um tipo especial de Morador.
1. **Atributos**: número (identificação na Urna), contagem de votos.
2. **Método**:
- **Construtor**: inicializa com nome e apartamento do Morador e atribui 0 à contagem de votos.


In [6]:
class Candidato(Morador):
    def __init__(self, nome, num_apartamento, numCandidatura):
        super().__init__(nome, num_apartamento)
        self.numCandidatura = numCandidatura
        self.votosRecebidos = 0